# Network flow optimization model

## Sets & Subsets

* $N$: Set of nodes in the network, i $\in$ {0,1,…,N} 

* $A$: Set of arcs in the network, k $\in$ {0,1,…,A}

* $S$ $\subset N$: Subset of nodes that represent the beginning of the network (source), s $\in$ {0,1,…,S}

* $Z$ $\subset N$: Subset of nodes that represent the end of the network (sink), z $\in$ {0,1,…,Z}

## Parameters

#### Node's parameters

* $l^{n}_{i}$: Is the lower storing capacity for node ${i}$, $\forall i \in N$

* $u^{n}_{i}$: Is the upper storing capacity for node ${i}$, $\forall i \in N$

* $c^{n}_{i}$: Is the unit cost for node ${i}$, $\forall i \in N$

* $\gamma^{n}_{i}$: Is the initial amount in the network for node ${i}$, $\forall i \in S \subset N$


#### Arc's parameters

* $l^{a}_{i,j}$: Is the lower storing capacity for arc ${(i,j)}$, $\forall (i,j) \in A$

* $u^{a}_{i,j}$: Is the upper storing capacity for arc ${(i,j)}$, $\forall (i,j) \in A$

* $c^{a}_{i,j}$: Is the unit transportation cost from node ${i}$ to node ${j}$, $\forall (i,j) \in A$

## Decision variables

#### Node's decisions

* $\tau^{n}_{i}$: Is the amount to be stored in the node ${i}$, $\forall i \in N$

#### Arc's decisions

* $\upsilon^{a}_{i,j}$: Is the amount to be send from node ${i}$ to node ${j}$, $\forall (i,j) \in A$

## Objective function 
**Maximize total flow and minimize network costs**

$$ \ Min\space \ Z = \sum_{i \in N} c^{n}_{i}*\tau^{n}_{i} + \sum_{(i,j) \in A} c^{a}_{i,j}*\upsilon^{a}_{i,j} $$

$$ \ Max\space \ Z = \sum_{(i,j) \in A} \upsilon^{a}_{i,j} $$

## Constraints

#### Node's capacity
$$ l^{n}_{i} \leq  \tau^{n}_{i} \leq u^{n}_{i}, \forall i \in N $$

#### Arc's capacity
$$ l^{a}_{i,j} \leq  \upsilon^{a}_{i,j} \leq u^{a}_{i,j}, \forall (i,j) \in A $$

#### Flow balancing equations
Flow balancing (except for initial and ending nodes):
$$ \sum_{(i,j) \in A} \upsilon^{a}_{i,j} - \sum_{(j,k) \in A} \upsilon^{a}_{j,k} = \tau^{n}_{j}, \forall j \in N - S \cup Z  $$
Initial nodes flow balance:
$$ \sum_{(s,j) \in A} \upsilon^{a}_{s,j} + \tau^{n}_{s} = \gamma^{n}_{s}, \forall s \in S \subset N$$
Ending nodes flow balance:
$$ \sum_{(i,z) \in A} \upsilon^{a}_{i,z} = \tau^{n}_{z}, \forall z \in Z \subset N$$

### Nonnegative constraints
$$ \upsilon^{a}_{i,j} \geq 0, \forall (i,j) \in A $$
$$ \tau^{n}_{i} \geq 0, \forall i \in N $$
